In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np

%load_ext autoreload
%autoreload 2

TRY GPU
DEVICE:GPU


In [2]:
x_init = np.random.randn(1,3).astype(np.float32)
x2_init = np.random.randn(3).astype(np.float32)
U_init = np.random.randn(3,3).astype(np.float32)
V_init = np.random.randn(3,3).astype(np.float32)
W_init = np.random.randn(3,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

In [3]:
x = DenseTensor(x_init)
W = DenseTensor(W_init)
m = DenseTensor(m_init)
out = x.dot(W).relu()
out = out.logsoftmax()
out = out.mul(m).add(m).sum()
out.backward()

out.cpu().data, x

dense shape grad; (1,)


(array([0.44739205], dtype=float32),
 <DenseTensor <GPUBuffer with shape (1, 3)> with grad <GPUBuffer with shape (1, 3)>>)

In [4]:
x2 = DenseTensor(x2_init)#.gpu()
W = SparseTensor(W_init)
out = W.dot(x2).relu().sum()

out.backward()

out.cpu().data, x

[[ 0.7481819  -0.2950969   0.05694252]
 [-0.62634534  0.32370427 -2.1899743 ]
 [-0.05886433 -0.5906388   0.31257847]]
dense shape grad;aval, xval: 0.75,1.98:0-0 
aval, xval: -0.63,1.98:0-3 
aval, xval: -0.06,1.98:0-6 
aval, xval: -0.30,-0.14:1-1 
aval, xval: 0.32,-0.14:1-4 
aval, xval: -0.59,-0.14:1-7 
aval, xval: 0.06,0.16:2-2 
aval, xval: -2.19,0.16:2-5 
aval, xval: 0.31,0.16:2-8 
SUM/NNZ: 1.53 3 
SUM/NNZ: -1.64 3 
SUM/NNZ: 0.01 3 
 (1,)
aval, xval: 0.75,1.00:0-0 
aval, xval: -0.30,1.00:0-3 
aval, xval: 0.06,1.00:0-6 
aval, xval: -0.63,0.00:1-1 
aval, xval: 0.32,0.00:1-4 
aval, xval: -2.19,0.00:1-7 
aval, xval: -0.06,1.00:2-2 
aval, xval: -0.59,1.00:2-5 
aval, xval: 0.31,1.00:2-8 
SUM/NNZ: 0.69 3 
SUM/NNZ: -0.89 3 
SUM/NNZ: 0.37 3 


NameError: name 'mf' is not defined

In [5]:
import numpy as np
import pyopencl as cl

mf = cl.mem_flags

In [6]:
dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

((4, 8), (8,))

In [7]:
def fill_sparse(mat, sparsity=0.1):
    indices = np.array(range(mat.shape[1]))
    nrows = int(mat.shape[1]*sparsity)
    for row in range(mat.shape[0]):
        lim = nrows #+ int(np.random.random()*3)
        mat[row][np.random.permutation(indices)[:lim]] = np.random.random(lim)
    return mat

a = fill_sparse(a, sparsity)
#b = fill_sparse(b, sparsity)

In [8]:
a

array([[0.58006623, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.67575598, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.63831137, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.68815713, 0.        , 0.        ]])

In [9]:
b

array([0.16530164, 0.09382474, 0.85456353, 0.22054254, 0.71927327,
       0.40074044, 0.7605858 , 0.7475268 ], dtype=float32)

In [10]:
x2_init.T

array([ 1.9839467 , -0.13796845,  0.16029729], dtype=float32)

In [11]:
mult = a.dot(b)
mult

array([0.0958859 , 0.14903294, 0.0598894 , 0.27577239])

In [12]:
mult.shape

(4,)

In [13]:
def to_data(mat):
    ellwidth = int(mat.shape[1]/2)
    all_rows = []
    all_idxs = []
    all_nnzs = []
    for row in range(mat.shape[0]):
        rowdata = []
        colidxs = []
        all_nnzs.append(0)
        for col in range(mat.shape[1]):
            val = mat[row][col]
            if val != 0:
                rowdata.append(val)
                colidxs.append(col)
                all_nnzs[-1] += 1
        rowdata = np.array(rowdata)
        rowdata.resize(ellwidth)
        all_rows.append(rowdata)
        colidxs = np.array(colidxs)
        colidxs.resize(ellwidth)
        all_idxs.append(colidxs)
    all_rows = np.array(all_rows).astype(np.float32).flatten()
    all_idxs = np.array(all_idxs).astype(np.uint32).flatten()
    all_nnzs = np.array(all_nnzs).astype(np.uint32)
    return all_rows, all_idxs, all_nnzs, ellwidth

In [14]:
adata, acols, annz, ellwa = to_data(a)
adata, acols, annz, ellwa

(array([0.5800662 , 0.        , 0.        , 0.        , 0.675756  ,
        0.        , 0.        , 0.        , 0.6383114 , 0.        ,
        0.        , 0.        , 0.68815714, 0.        , 0.        ,
        0.        ], dtype=float32),
 array([0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0], dtype=uint32),
 array([1, 1, 1, 1], dtype=uint32),
 4)

In [15]:
#acols = acols.astype(np.uint32)
#annz = annz.astype(np.uint32)

In [16]:
adata, acols, annz, b

(array([0.5800662 , 0.        , 0.        , 0.        , 0.675756  ,
        0.        , 0.        , 0.        , 0.6383114 , 0.        ,
        0.        , 0.        , 0.68815714, 0.        , 0.        ,
        0.        ], dtype=float32),
 array([0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0], dtype=uint32),
 array([1, 1, 1, 1], dtype=uint32),
 array([0.16530164, 0.09382474, 0.85456353, 0.22054254, 0.71927327,
        0.40074044, 0.7605858 , 0.7475268 ], dtype=float32))

In [17]:
adata_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=annz)
b_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void SpMVNaive(__global  float* matData,     // INPUT MATRIX DATA
                        __global  uint*  colIdx,
                        __global  uint*  rowNnz,
                        uint   ellwidth,
                        __global  float* vector_x,    // INPUT
                        __global  float* vector_y    // OUTPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  
  
  uint nnz    = rowNnz[gid];
  float sum = 0;
  for (uint i = 0; i < nnz; i++) {
    uint index   = gid * ellwidth + i;
    uint col     = colIdx[index];
    float aval  = matData[index];
    float xval  = vector_x[col];
    printf("aval, xval: %.2f,%.2f:%i-%i \\n", aval, xval, col, index);
    sum  += aval * xval;
  }
  printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
  vector_y[gid] = sum;
}""").build()

In [18]:
a.shape, b.shape

((4, 8), (8,))

In [19]:
res = np.zeros(a.shape[0]).astype(np.float32)
res

array([0., 0., 0., 0.], dtype=float32)

In [20]:
rows = a.shape[0]

In [21]:
ellw = np.array([ellwa]).astype(np.uint32)
ellw

array([4], dtype=uint32)

In [22]:
mult = mult.astype(np.float32)

In [23]:
mult

array([0.0958859 , 0.14903294, 0.05988939, 0.2757724 ], dtype=float32)

In [24]:
mult.nbytes

16

In [25]:
len(mult)*4

16

In [26]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, mult.nbytes)
knl = prg.SpMVNaive  # Use this Kernel object for repeated calls
knl(queue, [rows,], None, adata_buf, acols_buf, annzs_buf, ellw, b_buf, res_buf)

res_np = np.empty_like(a)
cl.enqueue_copy(queue, res, res_buf)

aval, xval: 0.58,0.17:0-0 
aval, xval: 0.68,0.22:3-4 
aval, xval: 0.64,0.09:1-8 
aval, xval: 0.69,0.40:5-12 
SUM/NNZ: 0.10 1 
SUM/NNZ: 0.15 1 
SUM/NNZ: 0.06 1 
SUM/NNZ: 0.28 1 


In [27]:
res_buf

In [28]:
res

array([0.0958859 , 0.14903294, 0.0598894 , 0.2757724 ], dtype=float32)

In [29]:
mult

array([0.0958859 , 0.14903294, 0.05988939, 0.2757724 ], dtype=float32)

In [30]:
(res-mult).sum()

3.7252903e-09

## Weight update kernel

In [31]:
dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

((16,), (16,))

In [58]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, x.nbytes)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, x.nbytes)
y_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, x.nbytes)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, x.nbytes)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         __global  uint* xoutidx,    // INPUT
                         __global  float* yout,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  yout[gid] = y[gid];
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint idx = xoutidx[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool smaller = tempval > valx;
    bool smaller2 = tempval2 > valy;
      
    posx += (smaller)?1:0;
    posy += (smaller2)?1:0;
  }
  //printf("posx:%i", posx);
  xout[posx] = valx;
  xoutidx[posx] = gid;
  yout[posy] = valy;
  youtidx[posy] = gid;
}""").build()

In [59]:
knl = prg.genwupdate  # Use this Kernel object for repeated calls
knl(queue, [dim,], None, x_buf, y_buf, x_cp_buf, x_idx_buf, y_cp_buf, y_idx_buf)

resx = np.zeros(dim).astype(np.float32)
resxidx = np.zeros(dim).astype(np.uint32)
resy = np.zeros(dim).astype(np.float32)
resyidx = np.zeros(dim).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resy, y_cp_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [60]:
x

array([0.62714505, 0.89000505, 0.84990025, 0.5951234 , 0.3992664 ,
       0.44664982, 0.32236633, 0.81121296, 0.4800176 , 0.01496169,
       0.7488251 , 0.82636327, 0.99215335, 0.8140302 , 0.94315946,
       0.6669325 ], dtype=float32)

In [61]:
resx

array([0.99215335, 0.94315946, 0.89000505, 0.84990025, 0.82636327,
       0.8140302 , 0.81121296, 0.7488251 , 0.6669325 , 0.62714505,
       0.5951234 , 0.4800176 , 0.44664982, 0.3992664 , 0.32236633,
       0.01496169], dtype=float32)

In [62]:
resxidx

array([12, 14,  1,  2, 11, 13,  7, 10, 15,  0,  3,  8,  5,  4,  6,  9],
      dtype=uint32)

In [63]:
y

array([0.60745186, 0.5155412 , 0.4907725 , 0.21801807, 0.7653721 ,
       0.29214406, 0.09749307, 0.6866005 , 0.9963886 , 0.36705196,
       0.10319717, 0.19946894, 0.9527147 , 0.93879867, 0.6289892 ,
       0.53214246], dtype=float32)

In [64]:
resy

array([0.9963886 , 0.9527147 , 0.93879867, 0.7653721 , 0.6866005 ,
       0.6289892 , 0.60745186, 0.53214246, 0.5155412 , 0.4907725 ,
       0.36705196, 0.29214406, 0.21801807, 0.19946894, 0.10319717,
       0.09749307], dtype=float32)

In [65]:
resyidx

array([ 8, 12, 13,  4,  7, 14,  0, 15,  1,  2,  9,  5,  3, 11, 10,  6],
      dtype=uint32)

# OTHER

import numpy as np
import pyopencl as cl

mf = cl.mem_flags

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

In [66]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
val_out_buf = cl.Buffer(ctx, mf.READ_WRITE, 4*topk*topk)
x_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         uint topk,
                         __global  uint* xoutidx,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool larger = tempval > valx;
    bool larger2 = tempval2 > valy;
      
    posx += (larger)?1:0;
    posy += (larger2)?1:0;
  }
  //printf("posx:%i", posx);
  if (posx < topk) {
    xoutidx[posx] = gid;
  }
  if (posy < topk) {
    youtidx[posy] = gid;
  }
  if (gid < topk) {
    uint i = gid;
    for (uint j=0; j<topk; j++) {
      xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
    }
  }
}""").build()

In [67]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
event = knl(queue, [dim,], None, x_buf, y_buf, val_out_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

#event.wait()
val_out = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, val_out, val_out_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf, wait_for=[event])
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [68]:
val_out

array([0.9885703 , 0.94523907, 0.93143225, 0.7593665 , 0.93975335,
       0.8985619 , 0.88543683, 0.7218679 , 0.8867909 , 0.8479209 ,
       0.8355355 , 0.681185  , 0.8468309 , 0.80971247, 0.79788524,
       0.6504899 ], dtype=float32)

In [69]:
resxidx

array([12, 14,  1,  2], dtype=uint32)

In [70]:
resyidx

array([ 8, 12, 13,  4], dtype=uint32)

In [71]:
from __future__ import division

KERNEL_CODE = """
// Thread block size
#define BLOCK_SIZE %(block_size)d
// Matrix dimensions
// (chosen as multiples of the thread block size for simplicity)
#define WA %(w_a)d // Matrix A width
#define HA %(h_a)d // Matrix A height
#define WB %(w_b)d // Matrix B width
#define HB WA  // Matrix B height
#define WC WB  // Matrix C width
#define HC HA  // Matrix C height
/*
 * Copyright 1993-2009 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */
/* Matrix multiplication: C = A * B.
 * Device code.
 */
#define AS(j, i) As[i + j * BLOCK_SIZE]
#define BS(j, i) Bs[i + j * BLOCK_SIZE]
////////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! WA is A's width and WB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel __attribute__((reqd_work_group_size(16,16,1))) 
void
matrixMul( __global float* C, __global float* A, __global float* B)
{
    __local float As[BLOCK_SIZE*BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE*BLOCK_SIZE];
    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);
    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);
    // Index of the first sub-matrix of A processed by the block
    int aBegin = WA * BLOCK_SIZE * by;
    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + WA - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * WB;
    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;
    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {
        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + WA * ty + tx];
        BS(ty, tx) = B[b + WB * ty + tx];
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);
        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);
        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;
}
"""
